In [1]:
!pip install pyspark
!pip install -q findspark



     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 30.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9ab0a4e479dd8a53eb8b5ebaf7a545b3eeeaa65875b8f22474361edfdb863853
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [11]:
from pyspark import SparkConf, SparkContext
import os
sc = SparkContext.getOrCreate()
import time
from statistics import mean

path = "UF"
dir_list = os.listdir(path)
print(dir_list)
dir_list = dir_list
print(dir_list)

avg_time = []

counter = 0
for prof in dir_list:
  counter +=1
  if prof != '.ipynb_checkpoints':
    prof = prof.replace(',','')

    start = time.time()

    #Find prof's last name
    l_name = prof[:-4].split('_')[-1]

    file_exists = os.path.exists(f'{path}/{prof}')
    print(f'Scanning... {path}/{prof}')
    if file_exists == True:
      #Part 1: Find co author with most citaitons
      mydata = sc.textFile(f'{path}/{prof}')
      header = mydata.first()
      mydata1 = mydata.filter(lambda row: row != header)
      authors_citations = mydata1.map(lambda line: line.split(';')[1:]).map(lambda fields: (fields[1].replace(', ',','), fields[0])).flatMapValues(lambda v: v.split(','))

      # Can be combined into 1 to combine the filter and map
      author = authors_citations.filter(lambda x: l_name in x[1])#.collect()  #Prof Last name!
      #author = sc.parallelize(author)
      co_authors = authors_citations.subtract(author).map(lambda x: (x[1].replace(' ',''), int(x[0])))
      #co_authors.take(1)

      # Co-author count
      try:
        co_author_sum = co_authors.reduceByKey(lambda v1, v2: v1 + v2)
        co_author_order = co_author_sum.map(lambda x: (x[1], x[0])).sortByKey(False)

        #print(prof[:-4])
        for item in co_author_order.take(1):
          #print(item)
          print(f'{counter}: The best coauthor for {prof[:-4]} is {item[0]} with {item[1]} citations.')
      except:
        print('error, skipped professor')

    end = time.time()
    avg_time.append(end - start)

print('The average time to find the best co-author for each professor was: ' , mean(avg_time), 's after scanning', counter, 'professors.')




['Dimitri_Bourilkov.csv', 'Thomas_D_Schmittgen.csv']
['Dimitri_Bourilkov.csv', 'Thomas_D_Schmittgen.csv']
Scanning... UF/Dimitri_Bourilkov.csv
1: The best coauthor for Dimitri_Bourilkov is 149417 with ... citations.
Scanning... UF/Thomas_D_Schmittgen.csv
error, skipped professor
The average time to find the best co-author for each professor was:  0.637321949005127 s after scanning 2 professors.


In [4]:
from pyspark import SparkConf, SparkContext
import os
sc = SparkContext.getOrCreate()
import time
from statistics import mean

path = "UM"
dir_list = os.listdir(path)
print(dir_list)
dir_list = dir_list[1:]
print(dir_list)

avg_time = []

mean1 = []

counter = 0
for prof in dir_list:
  counter +=1
  if prof != '.ipynb_checkpoints':
    prof = prof.replace(',','')

    start = time.time()

    #Find prof's last name
    l_name = prof[:-4].split('_')[-1]

    file_exists = os.path.exists(f'{path}/{prof}')
    print(f'Scanning... {path}/{prof}')
    if file_exists == True:
      #Part 1: Find co author with most citaitons
      mydata = sc.textFile(f'{path}/{prof}')
      header = mydata.first()
      mydata1 = mydata.filter(lambda row: row != header)
      authors_citations = mydata1.map(lambda line: line.split(';')[1:]).map(lambda fields: (fields[1].replace(', ',','), fields[0])).flatMapValues(lambda v: v.split(','))

      # Can be combined into 1 to combine the filter and map
      author = authors_citations.filter(lambda x: l_name in x[1])#.collect()  #Prof Last name!
      #author = sc.parallelize(author)
      co_authors = authors_citations.subtract(author).map(lambda x: (x[1], 1))
      #co_authors.take(1)

      # Co-author count
      try:
        co_author_sum = co_authors.reduceByKey(lambda v1, v2: v1 + v2)
        co_author_order = co_author_sum.map(lambda x: (x[1], x[0])).sortByKey(False)
  
        #print(prof[:-4])
        for item in co_author_order.take(1):
          #print(item)
          print(f'{counter}: The coauthor with the most papers for {prof[:-4]} is {item[1]} with {item[0]} papers.')
          mean1.append(item[0])
      except:
        print('error, skipped professor')

    end = time.time()
    avg_time.append(end - start)

print('The average time to find the best co-author for each professor was: ' , mean(avg_time), 's after scanning', counter, 'professors.')

print('The average numbers of papers for the best co-author was', mean(mean1))


['Luca_Mezincescu.csv', 'J_Sunil_Rao.csv', 'Amanda_Jensen-Doss.csv', 'Ben_Kirtman.csv', 'Elizabeth_Crocco.csv', 'Andres_M_Kanner.csv', 'Xiaodong_Cai.csv', 'Peter_Wysocki.csv', 'Joseph_M_Prospero.csv', 'Steven_A_Safren.csv', 'Antonello_Pileggi.csv', 'Michael_G_Brown.csv', 'Kaufui_Wong.csv', 'Arun_Malhotra.csv', 'Ross_Scalese.csv', 'James_Galvin.csv', 'Helen_Bramlett.csv', 'Peter_J_Minnett.csv', 'Wolfgang_Nonner.csv', 'Howard_A_Liddle.csv', 'Christine_Delgado.csv', 'Tan_a_Ince.csv', 'Rafael_I_Nepomechie.csv', 'Joshua_Gundersen.csv', 'Amjad_Farooq.csv', 'Jaime_R_Merchan.csv', 'TAKASHI_ASAI.csv', 'Amanda_J_Myers.csv', 'edison_trickett.csv', 'Shihab_Asfour.csv', 'Susan_E_Morgan.csv', 'Jay_S_Skyler.csv', 'savita_pahwa.csv', 'Dhananjay_Nanda.csv', 'Joel_Fishman.csv', 'Kenneth_G_Proctor.csv', 'Ricardo_J_Komotar.csv', 'Daniel_Messinger.csv', 'Philip_Rosenfeld.csv', 'Adam_Wanner.csv', 'Terri_Scandura.csv', 'Adrien_Eshraghi.csv', 'William_Johns.csv', 'Xi_(Steven)_Chen.csv', 'Vittorio_porciatti.cs